[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EnergyDerivatives/EnergyDerivatives.ipynb)

# EnergyDerivatives

Energy derivatives: payoff diagrams and Greeks (Delta, Gamma, Vega, Theta) using Black-Scholes.

**Output:** `deriv_payoff_diagrams.pdf`, `deriv_greeks_curves.pdf`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
from scipy.stats import norm

# --- Black-Scholes helper functions ---

def bs_d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def bs_d2(S, K, T, r, sigma):
    return bs_d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

def bs_call_price(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    d2 = bs_d2(S, K, T, r, sigma)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    d2 = bs_d2(S, K, T, r, sigma)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def bs_delta_call(S, K, T, r, sigma):
    return norm.cdf(bs_d1(S, K, T, r, sigma))

def bs_delta_put(S, K, T, r, sigma):
    return bs_delta_call(S, K, T, r, sigma) - 1.0

def bs_gamma(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def bs_vega(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return S * norm.pdf(d1) * np.sqrt(T) / 100  # per 1% move

def bs_theta_call(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    d2 = bs_d2(S, K, T, r, sigma)
    term1 = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
    term2 = -r * K * np.exp(-r * T) * norm.cdf(d2)
    return (term1 + term2) / 365  # per day


In [ ]:
# --- Payoff diagrams: Long Call, Long Put, Long Futures ---

K = 100
premium_call = 8
premium_put = 6

S_range = np.linspace(60, 140, 500)

# Payoffs
call_payoff = np.maximum(S_range - K, 0) - premium_call
put_payoff = np.maximum(K - S_range, 0) - premium_put
futures_payoff = S_range - K

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

titles = ['Long Call', 'Long Put', 'Long Futures']
payoffs = [call_payoff, put_payoff, futures_payoff]

for ax, title, payoff in zip(axes, titles, payoffs):
    ax.plot(S_range, payoff, color=COLORS['blue'], lw=2)
    ax.fill_between(S_range, payoff, 0, where=(payoff > 0),
                    color=COLORS['green'], alpha=0.3, label='Profit')
    ax.fill_between(S_range, payoff, 0, where=(payoff < 0),
                    color=COLORS['red'], alpha=0.3, label='Loss')
    ax.axhline(0, color='black', lw=0.8, ls='--')
    ax.axvline(K, color=COLORS['gray'], lw=0.8, ls=':', label=f'Strike K={K}')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Spot Price at Expiry')
    ax.set_ylabel('Profit / Loss')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

fig.suptitle('Energy Derivative Payoff Diagrams', fontsize=15, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'deriv_payoff_diagrams.pdf')
plt.show()


In [ ]:
# --- Greeks curves: Delta, Gamma, Vega, Theta ---

S = np.linspace(60, 140, 500)
K = 100
T = 0.25
r = 0.04
sigma = 0.35

delta_c = np.array([bs_delta_call(s, K, T, r, sigma) for s in S])
gamma_v = np.array([bs_gamma(s, K, T, r, sigma) for s in S])
vega_v = np.array([bs_vega(s, K, T, r, sigma) for s in S])
theta_v = np.array([bs_theta_call(s, K, T, r, sigma) for s in S])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

greek_data = [
    (axes[0, 0], 'Delta (Call)', delta_c, COLORS['blue']),
    (axes[0, 1], 'Gamma', gamma_v, COLORS['green']),
    (axes[1, 0], 'Vega (per 1%)', vega_v, COLORS['purple']),
    (axes[1, 1], 'Theta (per day)', theta_v, COLORS['red']),
]

for ax, title, values, color in greek_data:
    ax.plot(S, values, color=color, lw=2)
    ax.fill_between(S, values, alpha=0.15, color=color)
    ax.axvline(K, color=COLORS['gray'], lw=0.8, ls=':', label=f'Strike K={K}')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Spot Price (S)')
    ax.set_ylabel(title.split('(')[0].strip())
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

fig.suptitle(f'Black-Scholes Greeks  (K={K}, T={T}, r={r}, $\sigma$={sigma})',
             fontsize=15, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'deriv_greeks_curves.pdf')
plt.show()
